In [0]:
from pyspark.sql import functions as F

df_maiores = (
    df.groupBy("regiao")
      .agg(F.max("consumo_por_m3").alias("maior_consumo_m3"))
)

df_unicos = df_maiores.dropDuplicates(["maior_consumo_m3"])

df_top10_consumo = (
    df_unicos.orderBy(F.desc("maior_consumo_m3"))
             .limit(10)
)

df_top10_consumo_formatado = (
    df_top10_consumo
        .withColumn(
            "maior_consumo_m3",
            F.regexp_replace(
                F.regexp_replace(F.format_number("maior_consumo_m3", 2), ",", "X"),
                "\\.", ","
            )
        )
        .withColumn(
            "maior_consumo_m3",
            F.regexp_replace("maior_consumo_m3", "X", ".")
        )
)

display(df_top10_consumo_formatado)

df_top10_consumo_formatado.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("classes.gold.top10_regioes_maior_consumo_m3_FORMATADO")
